In [18]:
import pandas as pd

# Load the dataset
file_path = '../data/products_books_v1_10_10.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())


from gensim.models import Word2Vec
import gensim
import string


           id                                      product_title  \
0  0002005018                                       Clara Callan   
1  0399135782                             The Kitchen God's Wife   
2  0440234743                                      The Testament   
3  0452264464               Beloved (Plume Contemporary Fiction)   
4  0609804618  Our Dumb Century: The Onion Presents 100 Years...   

                                       product_image  product_price  \
0  http://images.amazon.com/images/P/0002005018.0...            NaN   
1  http://images.amazon.com/images/P/0399135782.0...            NaN   
2  http://images.amazon.com/images/P/0440234743.0...            NaN   
3  http://images.amazon.com/images/P/0452264464.0...            NaN   
4  http://images.amazon.com/images/P/0609804618.0...            NaN   

                                        product_soup  product_tags  
0  Clara Callan Richard Bruce Wright HarperFlamin...           NaN  
1    The Kitchen God's Wif

In [19]:

# Preprocess text data
data['processed_soup'] = data['product_soup'].str.lower().str.translate(str.maketrans('', '', string.punctuation))

# Prepare data for Word2Vec model
sentences = [row.split() for row in data['processed_soup'].dropna()]

# Train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save the model for later use
model.save("../data/book_wordvec.model")



In [20]:
# Updated function to recommend books based on similar word vectors
def recommend_books_updated(input_text, top_n=5):
    input_text = input_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    vector = model.wv[input_text].mean(axis=0)
    similar_vectors = model.wv.similar_by_vector(vector, topn=top_n + 10)  # Retrieve more results to filter unique titles
    recommended_titles = []
    for book_vector in similar_vectors:
        similar_title = data.loc[data['processed_soup'].apply(lambda x: any(word in x for word in input_text)), 'product_title'].unique()
        for title in similar_title:
            if title not in recommended_titles and len(recommended_titles) < top_n:
                recommended_titles.append(title)
    return recommended_titles


# Test the updated function with the same book title
recommend_books_updated("Clara Callan")

['Clara Callan',
 'Cunt: A Declaration of Independence (Live Girls)',
 'Callander Square',
 'Cunt: A Declaration of Independence (Live Girls Series)']

In [21]:

# Test the updated function Kings the same book title
recommend_books_updated("Harry")

["The Sorcerer's Companion: A Guide to the Magical World of Harry Potter",
 "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
 "Harry Potter and the Sorcerer's Stone (Book 1)",
 'Harry Potter and the Chamber of Secrets (Book 2)',
 'Harry Potter and the Prisoner of Azkaban (Book 3)']

In [ ]:

# Test the updated function Kings the same book title
recommend_books_updated("Pale Kings and Prince")

['New Vegetarian: Bold and Beautiful Recipes for Every Occasion',
 'Prague : A Novel',
 'Seabiscuit: An American Legend',
 'Pigs in Heaven',
 'This Year It Will Be Different: And Other Stories']

In [ ]:

# Test the updated function Kings the same book title
recommend_books_updated("Kings")

['Pigs in Heaven',
 'Poisonwood Bible Edition Uk',
 'The Game of Kings (Lymond Chronicles, 1)',
 'The Bean Trees',
 'Homeland and Other Stories']

In [ ]:

# Test the updated function Kings the same book title
recommend_books_updated("Pale")

['Pale Fire: A Novel (Vintage International)',
 'On a Pale Horse',
 'Pale Horse Coming',
 'Pale Kings and Princes',
 'On a Pale Horse (Incarnations of Immortality, Bk. 1)']

In [4]:

from summa import keywords


In [8]:

# Input sentence
sentence = """He is the King"""

# Extract keywords using TextRank algorithm
keywords_list = keywords.keywords(sentence, words=3)

print("Keywords:", keywords_list)

Keywords: 


In [11]:
from gensim.summarization import keywords

# Example text
text = "Pale Kings and Prince"

# Getting keywords
key_words = keywords(text)

print(key_words)


In [12]:
from gensim.summarization import keywords

# Example text
text = "Pale Kings and Princes is a Spenser novel by Robert B. Parker. The title is taken from John Keats's poem La Belle Dame sans Merci: A Ballad. Following the murder of a reporter, Spenser is hired by a newspaper to investigate drug smuggling around the area of Wheaton, Massachusetts."""

# Getting keywords
key_words = keywords(text)

print(key_words)


drug
spenser
sans


In [7]:
import spacy

# Load the English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Your text
text = "Pale Kings and Prince"

# Process the text
doc = nlp(text)

# Extract nouns
nouns = [token.text for token in doc if token.pos_ == "NOUN"]

print("Nouns:", nouns)


Nouns: ['Kings']


In [14]:

def getMostSignificantKeyword(text):
    """
    Extracts keywords from the provided text, prioritizing parts of speech in a specific order.
    Continues until the combined length of keywords is at least 3 characters.
    """
    doc = nlp(text)
    priorityKey = ""
    fullsearch = ""
    
    # Define the order of part of speech tags to search based on their relevance
    pos_priority = ["NOUN", "VERB", "ADJ", "PROPN", "ADV", "PRON", "ADP", "CCONJ", "SCONJ", "DET", "AUX", "NUM", "PART", "INTJ", "SYM", "PUNCT", "X"]
    
    # Iterate over each part of speech in priority order
    for pos in pos_priority:
        keywords = [token.text for token in doc if token.pos_ == pos]
        for key in keywords:
            fullsearch += f" {key}"  # Append all found keywords to fullsearch
            if len(key) > len(priorityKey):
                priorityKey = key  # Update priorityKey if a longer keyword is found
            if len(priorityKey) >= 3:
                break  # Break inner loop if condition is met
        if len(priorityKey) >= 3:
            break  # Break outer loop if condition is met

    return priorityKey, fullsearch

In [15]:
getMostSignificantKeyword("Hi my name is john cena. I am the knight here")

('name', ' name')

In [16]:
getMostSignificantKeyword("Pale Kings and Prince")

('Kings', ' Kings')

In [22]:
getMostSignificantKeyword("Harry from Harry Potter and the Sorcerer's Stone (Book 1)")

('Harry', ' Harry')